
# wrangling.ipynb — Cleaning & Structuring

This notebook loads the **raw rankings CSV** from `acquisition.ipynb`, performs cleaning, type conversion,
feature engineering, and validation, and outputs a **clean pandas DataFrame**.


In [ ]:

import re
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

RAW_CSV = Path("./data_raw/rankings_raw.csv")
CLEAN_DIR = Path("./data_clean")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_CSV = CLEAN_DIR / "rankings_clean.csv"


In [ ]:

def load_raw(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype=str, keep_default_na=False)
    # Strip whitespace
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    return df


In [ ]:

def coerce_rank(val: str) -> float | np.nan:
    """Convert rank strings like '101-150', '1', 'N/A', '—' to numeric (use start of range).
    Returns NaN when not parseable.
    """
    if not isinstance(val, str) or val.strip() == "":
        return np.nan
    s = val.strip()
    # Remove dagger/footnote † or other stray symbols
    s = re.sub(r"[†‡*]+", "", s)
    # Ranges like '101-150' -> 101
    m = re.match(r"^(\d+)[\-–]\d+$", s)
    if m:
        return float(m.group(1))
    # Plain integers
    m = re.match(r"^\d+$", s)
    if m:
        return float(m.group(0))
    # N/A, -, etc.
    if s.lower() in {"n/a", "na", "-", "—"}:
        return np.nan
    # Fallback: extract first number
    m = re.search(r"\d+", s)
    return float(m.group(0)) if m else np.nan


In [ ]:

def coerce_score(val: str) -> float | np.nan:
    """Convert score strings to float; strip non-numeric except decimal point."""
    if not isinstance(val, str) or val.strip() == "":
        return np.nan
    s = val.strip()
    s = re.sub(r"[†‡*%]+", "", s)  # remove footnotes and % if present
    s = re.sub(r"[^0-9.]+", "", s) # keep digits and dot
    try:
        return float(s) if s != "" else np.nan
    except ValueError:
        return np.nan


In [ ]:

# Region mapping (simple heuristic — extend as needed)
REGION_MAP = {
    "United States": "North America", "Canada": "North America", "Mexico": "North America",
    "United Kingdom": "Europe", "Germany": "Europe", "France": "Europe", "Italy": "Europe", "Spain": "Europe",
    "Netherlands": "Europe", "Sweden": "Europe", "Norway": "Europe", "Finland": "Europe", "Denmark": "Europe",
    "Switzerland": "Europe", "Belgium": "Europe", "Ireland": "Europe", "Portugal": "Europe", "Austria": "Europe",
    "Russia": "Europe", "Poland": "Europe", "Czech Republic": "Europe", "Greece": "Europe", "Hungary": "Europe",
    "Turkey": "Asia", "China": "Asia", "Japan": "Asia", "South Korea": "Asia", "India": "Asia",
    "Singapore": "Asia", "Hong Kong": "Asia", "Taiwan": "Asia", "Malaysia": "Asia", "Thailand": "Asia",
    "Philippines": "Asia", "Indonesia": "Asia", "Pakistan": "Asia", "Bangladesh": "Asia", "United Arab Emirates": "Asia",
    "Saudi Arabia": "Asia", "Qatar": "Asia", "Israel": "Asia", "Iran": "Asia", "Lebanon": "Asia",
    "Australia": "Oceania", "New Zealand": "Oceania",
    "Brazil": "South America", "Argentina": "South America", "Chile": "South America", "Colombia": "South America", "Peru": "South America",
    "South Africa": "Africa", "Egypt": "Africa", "Nigeria": "Africa", "Kenya": "Africa", "Uganda": "Africa", "Ghana": "Africa", "Ethiopia": "Africa", "Morocco": "Africa", "Algeria": "Africa", "Tunisia": "Africa",
}


In [ ]:

def wrangle(df: pd.DataFrame) -> pd.DataFrame:
    # Standardize columns
    cols = {c.lower().strip(): c for c in df.columns}
    # Try flexible access
    def get_col(*names):
        for n in names:
            for c in df.columns:
                if c.strip().lower() == n.lower():
                    return c
        return None

    col_year = get_col("year")
    col_rank = get_col("rank", "world rank")
    col_name = get_col("university", "institution", "name")
    col_country = get_col("country")
    col_score = get_col("overall_score", "overall score", "score", "overall")

    # Minimal subset
    out = pd.DataFrame({
        "Year": df[col_year] if col_year else np.nan,
        "Rank": df[col_rank] if col_rank else np.nan,
        "University": df[col_name] if col_name else df.iloc[:, 0],
        "Country": df[col_country] if col_country else np.nan,
        "Overall_Score": df[col_score] if col_score else np.nan,
    }).copy()

    # Missing data handling for Rank
    out["Rank_num"] = out["Rank"].apply(coerce_rank)

    # Strategy: drop rows with missing rank (justify in report)
    before = len(out)
    out = out[~out["Rank_num"].isna()].copy()
    after = len(out)
    dropped = before - after
    print(f"Dropped {dropped} rows with missing/unparseable rank.")

    # Convert Year to int if possible
    out["Year"] = pd.to_numeric(out["Year"], errors="coerce").astype("Int64")

    # Score conversion
    out["Overall_Score_num"] = out["Overall_Score"].apply(coerce_score)

    # Feature engineering: Global_Region
    out["Global_Region"] = out["Country"].map(REGION_MAP).fillna("Other/Unknown")

    # Normalization: scale Overall_Score_num to [0,1]
    scaler = MinMaxScaler()
    score_vals = out[["Overall_Score_num"]].fillna(0.0)  # treat missing as 0 for scaling
    out["Overall_Score_Normalized"] = scaler.fit_transform(score_vals)

    # Primary key uniqueness: (University, Year)
    out["University"] = out["University"].astype(str).str.strip()
    out = out.drop_duplicates(subset=["University", "Year"], keep="first")
    dup_check = out.duplicated(subset=["University", "Year"], keep=False).sum()
    assert dup_check == 0, "Duplicate (University, Year) keys remain!"

    # Final tidy types
    out = out.rename(columns={
        "Rank_num": "Rank_int",
        "Overall_Score_num": "Overall_Score_float"
    })
    # Order columns
    final_cols = [
        "Year", "University", "Country", "Global_Region",
        "Rank_int", "Overall_Score_float", "Overall_Score_Normalized",
        "Rank", "Overall_Score"  # keep original raw strings for traceability
    ]
    out = out[final_cols]

    return out


In [ ]:

# ---- Run wrangling ----
raw = load_raw(RAW_CSV)
print(f"Raw shape: {raw.shape}")
clean = wrangle(raw)
print(f"Clean shape: {clean.shape}")
clean.to_csv(CLEAN_CSV, index=False)
clean.head()
